# Natural Language Processing

## Marco teórico
El procesamiento de lenguaje natural, mencionado como NLP a partir de ahora por sus siglas en inglés (Natural Language Processing), Es una rama de la inteligencia artficial que busca realizar análisis y transformaciones a cuerpos de texto para encontrar patrones, inferir significados y relaciones entre elementos (palabras, párrafos, oraciones, etc.) o, en general, servir en un modelo con algún propósito de toma de decisiones, inferencia o predicción.

Aquí veremos los básicos de esta área utilizando el paquete `TextAnalysis` en Julia.

### Documentos
Los documentos se definen como cualquier cuerpo de texto que puede ser representado de alguna manera específica en el disco de la computadora. Existen los siguientes tipos:
- Tipo archivo (FileDocument): Un documento representado como texto plano en el disco.
- Tipo string (StringDocument): Un documento representado como un string codificado en UTF8 en memoria RAM
- Tipo Token: (TokenDocument): Un documento representado como una sucesión de tokens UTF8, es decir, palabras o símbolos individuales (strings tras ser 'tokenizados').
- Tipo N-grama: (NGramDocument): Un documento representado como una colección de pares donde un elemento es un token y el otro es un entero que representa el una frecuencia de ocurrencia de dicho string.

Observemos los tipos continuación:


In [410]:
using TextAnalysis


#### Documentos de tipo string/cadena
Los documentos de tipo cadena suelen ser oraciones individuales, párrafos o textos más completos. No obstante, parte de tener una conjunto de datos limpio yace en organizar los textos en archivo o más pequeño posible.

Además, éstos se guardan en memoria ram, representados como una cadena de bits en codificación UTF-8. Por ello, tener textos muy grandes localizados en una sola variable identificadora puede dificultar su manipulación.


In [411]:
str = "Este es un texto de prueba. Este es la segunda oración"


In [412]:
sd = StringDocument(str)


#### Documentos de tipo archivo

Los documentos de tipo archivo se utilizan cuando tenemos un archivo contenedor del texto que queremos analizar. Para generarlo en julia basta con poner la ruta hacia el archivo:


In [413]:
pathname = "./nlp/archivo.txt"


In [414]:
fd = FileDocument(pathname)


#### Documentos de tipo token
Los token en el contexto de procesamiento de lenguaje natural son elementos individuales y uniformes que yacen en una colección. Se habla de token usualmente para referirse a palabras individuales cuando tenemos un alfabtero latino, no obstante, para otros alfabetos puede que el concepto cambie ligeramente.

Además, los tokens podrían referirse a letras individuales, oraciones o cualquier forma de agrupar el texto que posea información estructural. No obstante, esas dos agrupaciones mencionadas no suelen poseer información útil al ser respectivamente muy pequeñas y muy grandes.

Para crearlas en Julia podemos crear un arreglo de ellas:



In [415]:
mis_tokens = String["Esta", "es", "una", "oración", "de", "prueba"]


Aquí aprovechamos a mostrar además que para crear arreglos de un tipo uniforme en Julia, podemos anteponer el nombre del tipo y el compilador sabrá que debe esperar y forzar dicho tipo (ejemplo, `Int32` en un arreglo forzaría a todos los `Integer` dentro a que sean representado por 32 bits)


In [416]:
typeof(mis_tokens)


In [417]:
td = TokenDocument(mis_tokens)


#### Documento de tipo N-grama
Podemos pensar en que el documento de tipo token puede ser generado a partir de un documento de tipo string, y así también el de tipo string ser generado a partir de uno de tipo archivo. Esto es correcto y existen métodos específicos para ello.

Por una parte, para pasar de un string a una lista de tokens podemos utilizar el paquete `WordTokenizer`, el cual tiene múltiples métodos de tokenización de alto rendimiento para procesar grandes cuerpos de texto y reducirlos a tokens listos para el análisis.

Ahora, el documento de tipo N-grama tiene más sentido pensarlo viniendo de uno de tipo token, pues al tokenizar un texto grande, es muy probable que tengamos palabras repetidas y obtener la frecuencia en la que éstas palabras ocurren a lo largo de dicho texto juega un rol en el análisis exploratorio inicial.

Los documento de tipo N-grama son precisamente pares de tokens con un número entero que cuenta las ocurrencias de dicho token en algún texto. Aquí podemos generarlo de las siguientes maneras:



In [418]:
dict_ocurrencias = Dict("hola" => 1, "mundo" => 1)


In [419]:
ngd = NGramDocument(dict_ocurrencias)


## Procesamiento 
### Funciones
La siguiente es una exploración a algunas funciones ya definidas (para todos los tipos anteriores mediante multiple dispatch) en el paquete.


#### Texto


In [420]:
text(sd), text(fd), text(td)


#### Tokens


In [421]:
tokens(sd)


In [422]:
 tokens(fd)


#### N-gramas


In [423]:
ngrams(sd)


In [424]:
ngrams(fd)


In [425]:
ngrams(sd, 2)


In [426]:
ngrams(sd, 2, 3)


Podemos también extraer la estructura de un documento de N-gramas para entender si tiene bigramas o trigramas, etc.


In [427]:
ngram_complexity(NGramDocument(ngrams(sd), 2))


#### Metadata


In [428]:
language(sd) ## El lenguaje por defecto es inglés...


Podemos cambiarlo utilizando la versión 'mutadora' de la función `language`:


In [429]:
language!(sd, TextAnalysis.Languages.Spanish())


Así igual los demás elementos de la metadata...


In [430]:
title(sd), author(sd), timestamp(sd)


In [431]:
title!(sd, "Mi título"), author!(sd, "Yo"), timestamp!(sd, "Desconocido")


In [432]:
sd


### Procesamiento de documentos
Antes de comenzar a hacer transformaciones y análisis a los documentos, nos puede interesar hacer una limpieza en caso de tener caracteres corruptos o pequeñas molestias de formato que nos haría más limpio nuestro trabajo si no estuvieran


In [433]:
remove_corrupt_utf8!(sd) ## <- ejemplo de ello


In [434]:
str_2 = StringDocument("HolA!!!,. Soy un teXto, que No está mUy Bien escrito..")


In [435]:
prepare!(str_2, strip_punctuation)


In [436]:
text(str_2)


Removiendo las mayúsculas...


In [437]:
remove_case!(str_2)


In [438]:
text(str_2)


Podemos además remover ciertas palabras..


In [439]:
remove_words!(str_2, [" no"])


In [440]:
text(str_2)


Otras posibilidades son:

```
- prepare!(sd, strip_articles)
- prepare!(sd, strip_indefinite_articles)
- prepare!(sd, strip_definite_articles)
- prepare!(sd, strip_preposition)
- prepare!(sd, strip_pronouns)
- prepare!(sd, strip_stopwords)
- prepare!(sd, strip_numbers)
- prepare!(sd, strip_non_letters)
- prepare!(sd, strip_spares_terms)
- prepare!(sd, strip_frequent_terms)
- prepare!(sd, strip_html_tags)
```

Además de poder ser utilizadas juntas:

`prepare!(sd, strip_articles| strip_numbers| strip_html_tags)`

